# CALFEWS: California Food-Energy-Water System simulation model
This notebook provides an introductory tutorial for how to run the California Food-Energy Water System (CALFEWS) simulation model. The major sections are as follows:
1. Setting simulation parameters
1. Using different hydrologic inputs
1. Running the model
1. Analyzing and visualizing model output

## Setting simulation parameters
The main parameters file for CALFEWS is ``runtime_params.ini``. This file allows the user to change the type of simulation, the input data, output data characteristics, etc. Be sure to follow the instructions at the top of the file regarding syntax - failing to do so may result in improper parameter parsing.

The most important parameters are:
1. `model_mode`: This defines which type of simulation is run. There are four defined modes at present:
    1. `validation`: Validation mode runs over the historical time period for which good data exists (water years 1997-2016) and incorporates major changes to environmental regulations and water supply infrastructure over that period. This is used to validate model outputs against historical reservoir storage, delta pumping, groundwater bank accounts, etc. 
    1. `simulation`: Simulation mode takes a particular "scenario" of environmental regulations and infrastructure, and holds them fixed over an entire simulation. The scenario will be discussed below under `scenario_name`. This can be carried out for any input time series, such as the longer 111-year reconstructed dataset used by Zeff et al. (2020). Simulation mode is useful for risk-based analyses under present or future conditions.
    1. `sensitivity`: Sensitivity mode runs a sensitivity analysis over uncertain model variables such as irrigation demand or the stringency of environmental regulations. Sensitivity mode can dispatch many independent simulations in parallel on HPC resources. However, *sensitivity mode is not fully finished at present*.
    1. `climate_ensemble`: Climate ensemble mode runs an ensemble of present and future climate model projections through CALFEWS. This mode can also take advantage of parallelized dispatch.
1. `flow_input_type` and `flow_input_source`: These parameters dictate which input data is used for the model, for all modes except validation, which always uses the historical dataset. These parameters will be discussed in the "Using different hydrologic inputs" section below.
1. `scenario_name`: The scenario name dictates the environmental and infrastructure scenario that is used for all modes other than Validation mode (which uses the historical regulations and infrastructure). The default (used by Zeff et al. (2020)) is conditions as of water year 2017. The only other scenario currently offered is ``FKC_capacity_rehab_full``, in which the Friant-Kern Canal, which has seen reduced conveyance capacity over time due to subsidence, is restored to its original capacity. Other scenarios will be available soon.
1. `short_test`: To run a short test, this parameter can be set to the number of days that you would like to run (e.g., 365). If set to -1, the entire simulation will be run.
1. `output_directory`: The directory where model output will be stored. Default is the "results" folder, but this can be changed. Note that the path should be relative to the repository base directory.
1. `print_log`: If True, print statements, errors, etc., will be output to a text file. Else it defaults to the terminal.
1. `clean_output`: If True, only non-zero time series will be included in the HDF5 model output. If False, all time series will be included, even if zero at all dates.
1. `save_full`: Save Model objects from simulation as Pickle file, in addition to the HDF5 output? This should be turned off for sensitivity analyses due to their size. But at least one simulation's output is necessary for visualizing model output.
1. `parallel_mode`: Run in parallel (True) or serial (False)?
1. `cluster_mode`: Run on HPC cluster, or single machine? Only matters if running in parallel.
1. `num_nodes`: Number of nodes to use (only matters if running on a cluster in parallel).
1. `processors_per_node`: Number of processors per node (only matters if running on a cluster in parallel).
1. `num_processors`: Number of processors to use on computer (only matters if running in parallel but NOT on a cluster).

See `runtime_params_validation.ini` for an example of how to run the validation experiment from Zeff et al. (2020) on a laptop.

In [1]:
### Uncomment the next line to print the file contents
# ! cat runtime_params_validation.ini

## Using different hydrologic inputs
The CALFEWS model converts hydrologic inputs into output time series such as reservoir storage, pumping from the Sacramento-San Joaquin Delta, deliveries of state and federal project water to irrigation districts, and groundwater banking accounts. For validation mode, the inputs include historical snow, streamflow, precipitation, evaporation, and urban water demand observations over the 1997-2016 water years. This data can be found in ``calfews_src/data/input/calfews_src-data.csv``.

In [2]:
import pandas as pd
historical = pd.read_csv('calfews_src/data/input/calfews_src-data.csv')
historical.head()

,datetime,realization,ORO_snow,SHA_snow,FOL_snow,YRS_snow,MHB_snow,PAR_snow,NHG_snow,NML_snow,...,SLF_out,CCC_pump,BRK_pump,HRO_pump,TRP_pump,delta_inflow,vernalis_inflow,EDM_pump,LAP_pump,SOB_pump
0,10/1/1996,1,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,...,605.24,204,44,6524,4122,17788.0,2110,3097,325,184
1,10/2/1996,1,0.0,0.0,0.0,0.0,0.12,0.12,0.12,0.36,...,605.24,213,43,6524,4374,17538.0,2120,2548,377,176
2,10/3/1996,1,0.0,0.0,0.0,0.0,0.24,0.24,0.24,0.36,...,605.24,206,46,6518,4374,17667.0,2160,2357,58,191
3,10/4/1996,1,0.0,0.0,0.0,0.0,0.24,0.24,0.24,0.75,...,605.24,191,57,5885,4259,17307.0,2140,2347,409,208
4,10/5/1996,1,0.0,0.0,0.0,0.0,0.44,0.44,0.44,1.42,...,605.24,184,43,6538,4252,17540.0,2140,2217,370,199


For model modes other than validation, the user can specify which input time series they would like to use. As mentioned previously, this is done using the ``flow_input_type`` and ``flow_input_source`` parameters from ``runtime_params.ini``. The mapping between these two parameters and the input information is given in the json file ``calfews_src/data/input/base_inflows.json``. This file consists of a nested set of dictionaries. The outer set of keys (e.g., ``flow_input_file``, ``export_series``, etc.) contain the names of important variables that will be used within the model. The next layer's key corresponds to the ``flow_input_type``, and the third layer consists of the ``flow_input_source``. CALFEWS will choose the value corresponding to the two keys as listed in ``runtime_params.ini``. 

In [3]:
### Uncomment the next line to print the file contents
# ! cat calfews_src/data/input/base_inflows.json

As an example, the simulation mode run from Zeff et al. (2020) uses ``flow_input_type = "observations"`` and ``flow_input_source = "CDEC"``. With these keys, CALFEWS will choose ``calfews_src/data/input/calfews_src-data-sim.csv`` as the ``flow_input_file``. Other keys within the json file provide metadata such as the simulation start and end dates, whether the data contain leap days, and the flow units. Here are some of the main combinations, and the source each dataset:

| flow_input_type | flow_input_source | model_mode | File (relative to calfews_src/data/) | Water Years | Source |
| :--             | :--               | :--        | :--  | :--         | :--    |
| observations    | CDEC              | validation | input/calfews_src-data-sim.csv | 1997-2016   | California Data Exchange Center (CDEC). See Zeff et al. (2020), In Prep, for details.|
| observations    | CDEC              | simulation | input/calfews_src-data.csv | 1906-2016   | California Data Exchange Center (CDEC), plus statistical reanalysis. See Zeff et al. (2020), In Prep, for details.|
| observations    | CDEC_wet_dry      | simulation | input/calfews_src-data-wet-dry.csv | 1997, 2014 | Same dataset as first row, but wet (1997) and dry (2014) water years manually spliced together. |
| downscaled_20062016    | WRF               | simulation | CA_FNF_WRF/WRF_20062016.csv | 2007-2017 | Output from WRF, from Holtzman et al. (2020), JAMES,  https:// doi.org/10.1029/2019MS001832 |
| downscaled_20062016    | WRF_GW_corr       | simulation | CA_FNF_WRF/WRF_GWCorrect_20062016.csv | 2007-2017 | Output from WRF with groundwater correction, from Holtzman et al. (2020), *JAMES*,  https:// doi.org/10.1029/2019MS001832 |
| downscaled_19502100    | canesm2_rcp45       | climate_ensemble | CA_FNF_climate_change/CA_FNF_canesm2_rcp45_r1i1p1.csv | 1950-2100 | CanESM2 output from CMIP5, then downscaled and bias corrected. 21 other model/RCP combinations also available. For details see Cohen et al. (2020), *JWRPM* (101061), In Press. |

## Running the model
### Check that all Python dependencies are met
If the Python imports below cause a problem, this means you do not have all of the requisite packages. Follow the instructions on the README to create a virtual environment and perform the installations. CALFWES requires Python 3. Also, be sure to check that the correct virtual environment is activated and being used for the Kernel in this notebook.

In [4]:
%load_ext cython

In [5]:
%%cython
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
import shutil
import sys
from configobj import ConfigObj
import json
from distutils.util import strtobool
import h5py
from calfews_src.model_cy cimport Model
from calfews_src.inputter_cy import Inputter
from calfews_src.scenario import Scenario
from calfews_src.util import *
from calfews_src.plotter import *
from calfews_src.visualizer import Visualizer

ValueError: calfews_src.model_cy.Model size changed, may indicate binary incompatibility. Expected 4856 from C header, got 1456 from PyObject

### Set model parameters
Before running the simulation, we need to be sure that the runtime parameters have been set appropriately. For this tutorial, we will run simulation mode with a short example time series that is two years long. This input has been created artificially by putting a very dry water year (2014) after a very wet water year (1997).

We start with the example runtime file for validation mode, then change ``model_mode`` to simulation and change ``flow_input_source`` to ``CDEC_wet_dry``. You can do this manually in a text editor, or in a terminal with the following commands:

In [ ]:
! cp runtime_params_validation.ini runtime_params.ini
! sed -i 's:\"validation\":\"simulation\":' runtime_params.ini
! sed -i 's:\"CDEC\":\"CDEC_wet_dry\":' runtime_params.ini

### Run the model
First, make sure you do not already have output stored at ``results/baseline_wy2017/simulation/CDEC_wet_dry/``. If so, it needs to be moved or deleted, or it will cause an error.

In [ ]:
### uncomment the next line to delete old results, or do it by hand in the File Explorer/Finder
! rm -rf results/baseline_wy2017/simulation/CDEC_wet_dry/

To run CALFEWS, simply run the script ``main.py`` from your favorite IDE. Alternatively, it can be run from the terminal using the command below. This should take about 2 minutes. 

In [ ]:
! python3 -W ignore run_main_cy.py

## Analyzing and visualizing model outputs
The output from the simulation we just ran can be found at ``results/baseline_wy2017/simulation/CDEC_wet_dry/results_CDEC_wet_dry.hdf5``. Results are stored in the HDF5 format, which is much more efficient for large files than using text files. The output can be viewed in a graphical interface using the [HDFView program](https://www.hdfgroup.org/downloads/hdfview/).

We can also interface with the HDF5 file directly using Python. Advanced data analysis and visualization functions can be found in the ``calfews_src/visualizer.py`` and ``calfews_src/sanker.py`` files. All figures displayed in Zeff et al. (2020) can be recreated by running the ``modeling_paper_notebook.ipynb`` Jupyter notebook. 

### Retrieving data
However, in this tutorial we will show you how to interface directly with the model output in order to explore the data and create plots of your own.

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import h5py
import json
from datetime import datetime
import matplotlib.pyplot as plt
from itertools import compress
import os
from datetime import datetime
from calfews_src import *
from calfews_src.visualizer import Visualizer

# results hdf5 file location from CALFEWS simulations
output_folder_sim = 'results/baseline_wy2017/simulation/CDEC_wet_dry/'
output_file_sim = output_folder_sim + 'results.hdf5'

# load Model objects from simulation for information about available time series
modelno_sim = pd.read_pickle(output_folder_sim + 'modelno0.pkl')
modelso_sim = pd.read_pickle(output_folder_sim + 'modelso0.pkl')


fig_folder = 'results/baseline_wy2017/simulation/figures/'

# now load simulation output
simulation = Visualizer(modelso_sim.district_list, modelso_sim.private_list, modelso_sim.city_list, modelso_sim.contract_list, modelso_sim.waterbank_list, modelso_sim.leiu_list)
simulation.get_results_sensitivity_number(output_file_sim, '', 10, 1995, 1)
datDaily = simulation.values

``datDaily`` is a dictionary with all of the results. Each element is a daily time series over the two year period. The data includes model input time series (e.g., inflow to Shasta Lake) as well as simulated model state variable outputs (e.g., releases from Shasta Lake, or pumping into the California Aqueduct, or State Water Project deliveries to Semitropic Water Storage District).

In [ ]:
## Uncomment line below to see a List all time series names
# print(list(datDaily.keys()))

Each of these keys can be interpreted using the spreadsheet ``results_key.xlsx``. This contatins 6 tabs: reservoirs, delta, contracts, districts, private, and waterbank. Each contains a list called ``Object List`` and a list called ``Object Attributes``. It also includes units for each variable. The column names for ``datDaily`` can be constructed by splicing members of each list together using an underscore (``_``). For example, the time series of releases (``R``) from Shasta Lake (``shasta``) is called ``shasta_R``.

To get water deliveries to each District or Private entity requires the additional step of specifying the contract name. So if we want the State Water Project (``swpdelta``) delivery (``*contract_delivery``) to Semitropic Water Storage District (``semitropic``), the time series will be called ``semitropic_swpdelta_delivery``.

Lastly, to get a District or Private entity's balance in a Water Banks, we specify the bank member's ``Object Key`` as the attribute. For example, the balance in the Kern Water Bank (``kwb``) owned by Semitropic (whose key is ``SMI``) will be called ``kwb_SMI``. We can also get the sum of all bank accounts in a particular bank with the attribute ``total`` (e.g., ``kwb_total``).

### Plotting data
Any time series can now be accessed at ``datDaily[*time_series_name]``. This allows us to plot any results of interest from Python.

The data includes important model inputs, such as snowpack (which tracks the annual maximum, and thus is non-decreasing over the water year) and reservoir inflows (note log scale):

In [ ]:
fig = plt.figure(figsize=(15,6))
ax = plt.subplot(121)
ax.plot(datDaily['shasta_SNPK'], label='Shasta')
ax.plot(datDaily['oroville_SNPK'], label='Oroville')
ax.plot(datDaily['millerton_SNPK'], label='Millerton')
ax.plot(datDaily['isabella_SNPK'], label='Isabella')
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Snowpack (inch)')
plt.xticks(rotation=315)

ax = plt.subplot(122)
ax.plot(datDaily['shasta_Q'], label='Shasta')
ax.plot(datDaily['oroville_Q'], label='Oroville')
ax.plot(datDaily['millerton_Q'], label='Millerton')
ax.plot(datDaily['isabella_Q'], label='Isabella')
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Inflow (taF/day)')
ax.set_yscale('log')
plt.xticks(rotation=315)

Additionally, we can look at model outputs such as reservoir storage and pumping from the Delta:

In [ ]:
fig = plt.figure(figsize=(15,6))
ax = plt.subplot(121)
ax.plot(datDaily['shasta_S'], label='Shasta')
ax.plot(datDaily['oroville_S'], label='Oroville')
ax.plot(datDaily['millerton_S'], label='Millerton')
ax.plot(datDaily['isabella_S'], label='Isabella')
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Storage (tAF)')
plt.xticks(rotation=315)

ax = plt.subplot(122)
ax.plot(datDaily['delta_HRO_pump'], label='Banks')
ax.plot(datDaily['delta_TRP_pump'], label='Tracy')
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Pumping (taF/day)')
# ax.set_yscale('log')
plt.xticks(rotation=315)

Here is cumulative water deliveries from various sources (left) and State Water Project deliveries to various districts:

In [ ]:
fig = plt.figure(figsize=(15,6))
ax = plt.subplot(121)
ax.plot(datDaily['swpdelta_contract'], label='SWP Table A')
ax.plot(datDaily['friant1_contract'], label='CVP Friant')
ax.plot(datDaily['cvpexchange_contract'], label='CVP Exchange')
ax.plot(datDaily['kingsriver_contract'], label='Kings')
ax.plot(datDaily['kernriver_contract'], label='Kern')
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Contract Deliveries (tAF)')
plt.xticks(rotation=315)

ax = plt.subplot(122)
ax.plot(datDaily['centralcoast_tableA_delivery'], label='Central Coast')
ax.plot(datDaily['losthills_tableA_delivery'], label='Lost Hills')
ax.plot(datDaily['semitropic_tableA_delivery'], label='Semitropic')
ax.plot(datDaily['wheeler_tableA_delivery'], label='Wheeler Ridge-Maricopa')
ax.plot(datDaily['socal_tableA_delivery'], label='SoCal')
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Contract Deliveries (tAF)')
plt.xticks(rotation=315)

Lastly, we can look at recharge and recovery from the Kern Water Bank

In [ ]:
fig = plt.figure(figsize=(15,6))
ax = plt.subplot(121)
ax.plot(datDaily['kwb_DLR'], label='Dudley Ridge')
# ax.plot(datDaily['kwb_KCWA'], label='KCWA')
ax.plot(datDaily['kwb_ID4'], label='KCWA ID4')
ax.plot(datDaily['kwb_SMI'], label='Semitropic')
ax.plot(datDaily['kwb_TJC'], label='Tejon-Castac')
ax.plot(datDaily['kwb_WRM'], label='Wheeler Ridge-Maricopa')
ax.plot(datDaily['kwb_WON'], label='Wonderful Company')
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Kern Water Bank balance (tAF)')
plt.xticks(rotation=315)